In [2]:
from langchain.llms import OpenAIChat
from langchain import LLMChain
from langchain.text_splitter import CharacterTextSplitter,TokenTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts.few_shot import FewShotPromptTemplate


import os
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
pd.set_option('display.max_colwidth', None)
from tqdm.notebook import tqdm
tqdm.pandas()
from ast import literal_eval
import random
import re

In [3]:
# !source ../../../elbaff_iesta_venv/bin/activate
# %pip install langchain
# %pip install python-dotenv
#%pip install openai

In [4]:
# Create a new OpenAI instance
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [5]:
text_splitter = CharacterTextSplitter()

In [6]:

basic_str = "Transform the following argument to an effective argument by maintaining the original length"
ideology_str = "for readers with a {ideology} political ideology"
content_str = "by preserving the content of the argument"
style_str = "by changing the style of the text only"

prompt_dict = {
    "basic": f"{basic_str}:",
    "ideology": f"{basic_str} {ideology_str}:",
    "content":  f"{basic_str} {content_str}:",
    "style":f"{basic_str} {style_str}:",
    "ideology-content": f"{basic_str} {ideology_str} {content_str}:",
    "ideology-style": f"{basic_str} {ideology_str} {style_str}:",
    "all": f"{basic_str} {ideology_str} {content_str} and {style_str}:",
}



In [7]:

def create_prompt_template(prompt):

    system_message_prompt = SystemMessagePromptTemplate.from_template(prompt)
    human_template="{ineffective_argument}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    return [system_message_prompt, human_message_prompt]

In [8]:

liberal_chat_prompt = ChatPromptTemplate.from_messages(create_prompt_template(prompt_dict["all"].format(ideology ="liberal")))
llm_chain = LLMChain(llm=chat, prompt=liberal_chat_prompt)

In [9]:
result = llm_chain.run(ineffective_argument="If there was no Kryptonite, can Superman defeat the Silver Surfer?")
print(result)

It's an interesting question to consider whether Superman could defeat the Silver Surfer if Kryptonite were not a factor. While the Silver Surfer possesses incredible cosmic powers, Superman's abilities are equally impressive. He has superhuman strength, speed, and agility, as well as the power of flight and heat vision. Additionally, Superman's moral code and determination to protect the innocent give him an edge in any battle. Ultimately, it's impossible to say for certain who would come out on top in a fight between these two iconic superheroes, but it's certainly a debate worth having.


In [10]:
cons_chat_prompt = ChatPromptTemplate.from_messages(create_prompt_template(prompt_dict["all"].format(ideology ="conservative")))
llm_chain = LLMChain(llm=chat, prompt=cons_chat_prompt)
result = llm_chain.run(ineffective_argument="If there was no Kryptonite, can Superman defeat the Silver Surfer?")
print(result)

It is a well-known fact that Superman is one of the most powerful superheroes in the comic book universe. However, some may wonder if he would be able to defeat the Silver Surfer if there was no Kryptonite involved. This is a valid question that has been debated among comic book fans for years.

On one hand, Superman possesses incredible strength, speed, and durability. He also has the ability to fly and shoot heat vision from his eyes. These powers have allowed him to defeat many formidable opponents in the past.

On the other hand, the Silver Surfer is also an incredibly powerful character. He possesses the Power Cosmic, which grants him a wide range of abilities including energy projection, matter manipulation, and the ability to travel faster than the speed of light. He has also proven to be a formidable opponent for other superheroes in the Marvel Universe.

Therefore, it is difficult to say for certain who would come out on top in a battle between Superman and the Silver Surfer. 

In [11]:
ineffective_argument = """
People should be allowed to do what they want with their own bodies. They aren't harming anyone, so there really is no reason to prevent them from having a good time. 2. There are more harmful drugs such as alcohol and tobacco which are legalized. If it's okay for those things to be consumed than it's certainly okay for the less harmful cannabis to be legalized. 3. If Cannabis is legalized it can be regulated. At this point there is no Fda or government oversight of the cannabis market.
"""
ideology = "conservative"
result_dict = {}
for k, prompt_template in prompt_dict.items():
    print(f"running {k} with prompt 'template {prompt_template}'")
    
    chat_prompt = ChatPromptTemplate.from_messages(create_prompt_template(prompt_template.format(ideology=ideology)))
    llm_chain = LLMChain(llm=chat, prompt=chat_prompt)
    result_dict[k] = llm_chain.run(ineffective_argument=ineffective_argument)
    #print(f"'{result}'\n\n")

    


running basic with prompt 'template Transform the following argument to an effective argument by maintaining the original length:'
running ideology with prompt 'template Transform the following argument to an effective argument by maintaining the original length for readers with a {ideology} political ideology:'
running content with prompt 'template Transform the following argument to an effective argument by maintaining the original length by preserving the content of the argument:'
running style with prompt 'template Transform the following argument to an effective argument by maintaining the original length by changing the style of the text only:'
running ideology-content with prompt 'template Transform the following argument to an effective argument by maintaining the original length for readers with a {ideology} political ideology by preserving the content of the argument:'
running ideology-style with prompt 'template Transform the following argument to an effective argument by ma

In [13]:
#%pip install redlines
#%pip install Markdown


In [15]:
for k, v in result_dict.items():
    print(f"\n{k} - {prompt_dict[k]}")
    diff = Redlines(ineffective_argument,v)
    display(Markdown(diff.output_markdown))


basic - Transform the following argument to an effective argument by maintaining the original length:


People should <span style="color:red;font-weight:700;text-decoration:line-through;">be </span><span style="color:red;font-weight:700;">have the freedom to make choices about their own bodies without interference. This includes the choice to use cannabis, which is less harmful than other legal substances like alcohol and tobacco. If these substances are </span>allowed to <span style="color:red;font-weight:700;text-decoration:line-through;">do what they want with </span><span style="color:red;font-weight:700;">be consumed, then it only makes sense to legalize cannabis as well. In fact, legalizing cannabis would allow for regulation and oversight by the government, which is currently lacking in the unregulated cannabis market. Therefore, it is time to legalize cannabis and give people the freedom to make </span>their own <span style="color:red;font-weight:700;text-decoration:line-through;">bodies. They aren't harming anyone, so there really is no reason to prevent them from having a good time. 2. There are more harmful drugs such as alcohol and tobacco which are legalized. If it's okay for those things to be consumed than it's certainly okay for the less harmful cannabis to be legalized. 3. If Cannabis is legalized it can be regulated. At this point there is no Fda or government oversight of the cannabis market.</span><span style="color:red;font-weight:700;">choices while also ensuring their safety through proper regulation.</span>


ideology - Transform the following argument to an effective argument by maintaining the original length for readers with a {ideology} political ideology:


People should <span style="color:red;font-weight:700;text-decoration:line-through;">be </span><span style="color:red;font-weight:700;">have the freedom to make choices about their own bodies without interference. This includes the choice to use cannabis, which is less harmful than other legal substances like alcohol and tobacco. If these substances are </span>allowed to <span style="color:red;font-weight:700;text-decoration:line-through;">do what they want with their own bodies. They aren't harming anyone, so there really </span><span style="color:red;font-weight:700;">be consumed, then it only makes sense to legalize cannabis as well. In fact, legalizing cannabis would allow for regulation and oversight by the government, which </span>is <span style="color:red;font-weight:700;text-decoration:line-through;">no reason to prevent them from having a good time. 2. There are more harmful drugs such as alcohol </span><span style="color:red;font-weight:700;">currently lacking in the unregulated cannabis market. By legalizing cannabis, we can ensure that it is used safely </span>and <span style="color:red;font-weight:700;text-decoration:line-through;">tobacco which are legalized. If it's okay for those things to be consumed than it's certainly okay for the less harmful cannabis to be legalized. 3. If Cannabis is legalized it can be regulated. At this point there is no Fda or government oversight of the cannabis market.</span><span style="color:red;font-weight:700;">responsibly.</span>


content - Transform the following argument to an effective argument by maintaining the original length by preserving the content of the argument:


...

<span style="color:red;font-weight:700;text-decoration:line-through;">People </span><span style="color:red;font-weight:700;">As individuals, we </span>should <span style="color:red;font-weight:700;text-decoration:line-through;">be allowed </span><span style="color:red;font-weight:700;">have the freedom </span>to <span style="color:red;font-weight:700;text-decoration:line-through;">do what they want with their </span><span style="color:red;font-weight:700;">make choices about our </span>own <span style="color:red;font-weight:700;text-decoration:line-through;">bodies. They aren't </span><span style="color:red;font-weight:700;">bodies without interference from the government. If we are not </span>harming <span style="color:red;font-weight:700;text-decoration:line-through;">anyone, so </span><span style="color:red;font-weight:700;">anyone else, </span>there <span style="color:red;font-weight:700;text-decoration:line-through;">really </span>is no reason to prevent <span style="color:red;font-weight:700;text-decoration:line-through;">them </span><span style="color:red;font-weight:700;">us </span>from <span style="color:red;font-weight:700;text-decoration:line-through;">having a good time. 2. There are </span><span style="color:red;font-weight:700;">enjoying ourselves. Additionally, it is hypocritical to allow the consumption of </span>more harmful <span style="color:red;font-weight:700;text-decoration:line-through;">drugs such as </span><span style="color:red;font-weight:700;">substances like </span>alcohol and tobacco <span style="color:red;font-weight:700;">while criminalizing the use of cannabis, </span>which <span style="color:red;font-weight:700;text-decoration:line-through;">are legalized. If it's okay </span><span style="color:red;font-weight:700;">is less harmful. Legalizing cannabis would also allow </span>for <span style="color:red;font-weight:700;text-decoration:line-through;">those things to be consumed than it's certainly okay for </span><span style="color:red;font-weight:700;">regulation and oversight by </span>the <span style="color:red;font-weight:700;text-decoration:line-through;">less harmful cannabis to be legalized. 3. If Cannabis </span><span style="color:red;font-weight:700;">FDA and government, which </span>is <span style="color:red;font-weight:700;text-decoration:line-through;">legalized it can be regulated. At this point there is no Fda or government oversight of </span><span style="color:red;font-weight:700;">currently lacking in </span>the <span style="color:red;font-weight:700;">unregulated </span>cannabis market.


ideology-style - Transform the following argument to an effective argument by maintaining the original length for readers with a {ideology} political ideology by changing the style of the text only:


<span style="color:red;font-weight:700;text-decoration:line-through;">People should be allowed </span><span style="color:red;font-weight:700;">Individuals have the right </span>to <span style="color:red;font-weight:700;text-decoration:line-through;">do what they want with </span><span style="color:red;font-weight:700;">make choices regarding </span>their own <span style="color:red;font-weight:700;text-decoration:line-through;">bodies. They aren't harming anyone, so </span><span style="color:red;font-weight:700;">bodies without interference. These choices do not harm others, and therefore should not be restricted. Additionally, it is worth noting that </span>there <span style="color:red;font-weight:700;text-decoration:line-through;">really is no reason to prevent them from having a good time. 2. There </span>are <span style="color:red;font-weight:700;text-decoration:line-through;">more harmful drugs </span><span style="color:red;font-weight:700;">other substances, </span>such as alcohol and <span style="color:red;font-weight:700;text-decoration:line-through;">tobacco </span><span style="color:red;font-weight:700;">tobacco, </span>which are <span style="color:red;font-weight:700;text-decoration:line-through;">legalized. </span><span style="color:red;font-weight:700;">legal despite being more harmful than cannabis. </span>If <span style="color:red;font-weight:700;text-decoration:line-through;">it's okay </span><span style="color:red;font-weight:700;">these substances are allowed, then it is reasonable to legalize cannabis, which is less harmful. Furthermore, legalization would allow </span>for <span style="color:red;font-weight:700;text-decoration:line-through;">those things to be consumed than it's certainly okay for </span><span style="color:red;font-weight:700;">regulation and oversight by </span>the <span style="color:red;font-weight:700;text-decoration:line-through;">less harmful cannabis to be legalized. 3. If Cannabis </span><span style="color:red;font-weight:700;">government, which </span>is <span style="color:red;font-weight:700;text-decoration:line-through;">legalized it can be regulated. At this point there is no Fda or government oversight of </span><span style="color:red;font-weight:700;">currently lacking in </span>the cannabis market.


all - Transform the following argument to an effective argument by maintaining the original length for readers with a {ideology} political ideology by preserving the content of the argument and by changing the style of the text only:


<span style="color:red;font-weight:700;text-decoration:line-through;">People should be allowed </span><span style="color:red;font-weight:700;">Individuals have the right </span>to <span style="color:red;font-weight:700;text-decoration:line-through;">do what they want with </span><span style="color:red;font-weight:700;">make decisions about </span>their own <span style="color:red;font-weight:700;text-decoration:line-through;">bodies. They aren't harming anyone, so there really </span><span style="color:red;font-weight:700;">bodies without interference. This includes the use of cannabis, which </span>is <span style="color:red;font-weight:700;text-decoration:line-through;">no reason to prevent them from having a good time. 2. There are more </span><span style="color:red;font-weight:700;">less </span>harmful <span style="color:red;font-weight:700;text-decoration:line-through;">drugs such as </span><span style="color:red;font-weight:700;">than other legal substances like </span>alcohol and <span style="color:red;font-weight:700;text-decoration:line-through;">tobacco </span><span style="color:red;font-weight:700;">tobacco. Legalizing cannabis would allow for regulation and oversight by the government, </span>which <span style="color:red;font-weight:700;text-decoration:line-through;">are legalized. If it's okay for those things to be consumed than it's certainly okay for </span><span style="color:red;font-weight:700;">is currently lacking in </span>the <span style="color:red;font-weight:700;text-decoration:line-through;">less harmful cannabis to be legalized. 3. If Cannabis is legalized it can be regulated. At this point there is no Fda or government oversight of the cannabis </span><span style="color:red;font-weight:700;">unregulated </span>market.

In [22]:
import itertools
for ptype1, ptype2 in itertools.combinations_with_replacement(prompt_dict.keys(), 2):
    if ptype1 == ptype2:
        continue
    print(f"\n{ptype1} VS. {ptype2}")
    print(len(f"{ptype1} VS. {ptype2}")*"-")

    diff = Redlines(result_dict[ptype1],result_dict[ptype2])
    display(Markdown(diff.output_markdown))


basic VS. ideology
------------------


People should have the freedom to make choices about their own bodies without interference. This includes the choice to use cannabis, which is less harmful than other legal substances like alcohol and tobacco. If these substances are allowed to be consumed, then it only makes sense to legalize cannabis as well. In fact, legalizing cannabis would allow for regulation and oversight by the government, which is currently lacking in the unregulated cannabis market. <span style="color:red;font-weight:700;text-decoration:line-through;">Therefore, </span><span style="color:red;font-weight:700;">By legalizing cannabis, we can ensure that </span>it is <span style="color:red;font-weight:700;text-decoration:line-through;">time to legalize cannabis </span><span style="color:red;font-weight:700;">used safely </span>and <span style="color:red;font-weight:700;text-decoration:line-through;">give people the freedom to make their own choices while also ensuring their safety through proper regulation.</span><span style="color:red;font-weight:700;">responsibly.</span>


basic VS. content
-----------------


<span style="color:red;font-weight:700;text-decoration:line-through;">People </span><span style="color:red;font-weight:700;">The argument for legalizing cannabis is a matter of personal freedom and responsibility. Individuals </span>should have the <span style="color:red;font-weight:700;text-decoration:line-through;">freedom </span><span style="color:red;font-weight:700;">right </span>to make choices about their own bodies without <span style="color:red;font-weight:700;text-decoration:line-through;">interference. This includes </span><span style="color:red;font-weight:700;">interference from </span>the <span style="color:red;font-weight:700;text-decoration:line-through;">choice to use cannabis, which </span><span style="color:red;font-weight:700;">government. Cannabis </span>is less harmful than other legal substances <span style="color:red;font-weight:700;text-decoration:line-through;">like </span><span style="color:red;font-weight:700;">such as </span>alcohol and <span style="color:red;font-weight:700;text-decoration:line-through;">tobacco. If these substances are allowed </span><span style="color:red;font-weight:700;">tobacco, so there is no reason for it </span>to <span style="color:red;font-weight:700;text-decoration:line-through;">be consumed, then it only makes sense to legalize cannabis as well. </span><span style="color:red;font-weight:700;">remain illegal. </span>In fact, legalizing cannabis would allow for regulation and oversight by the government, which is currently lacking in the unregulated cannabis market. Therefore, it is time <span style="color:red;font-weight:700;">for the government </span>to <span style="color:red;font-weight:700;">recognize the rights of individuals and </span>legalize <span style="color:red;font-weight:700;text-decoration:line-through;">cannabis and give people the freedom to make their own choices while also ensuring their safety through proper regulation.</span><span style="color:red;font-weight:700;">cannabis.</span>


basic VS. style
---------------


...

<span style="color:red;font-weight:700;text-decoration:line-through;">As individuals, we should </span><span style="color:red;font-weight:700;">Individuals </span>have the <span style="color:red;font-weight:700;text-decoration:line-through;">freedom </span><span style="color:red;font-weight:700;">right </span>to make choices <span style="color:red;font-weight:700;text-decoration:line-through;">about our </span><span style="color:red;font-weight:700;">regarding their </span>own bodies without <span style="color:red;font-weight:700;text-decoration:line-through;">interference from the government. If we are </span><span style="color:red;font-weight:700;">interference. These choices do </span>not <span style="color:red;font-weight:700;text-decoration:line-through;">harming anyone else, there is no reason to prevent us from enjoying ourselves. </span><span style="color:red;font-weight:700;">harm others, and therefore should not be restricted. </span>Additionally, it is <span style="color:red;font-weight:700;text-decoration:line-through;">hypocritical to allow the consumption of </span><span style="color:red;font-weight:700;">worth noting that there are other substances, such as alcohol and tobacco, which are legal despite being </span>more harmful <span style="color:red;font-weight:700;">than cannabis. If these </span>substances <span style="color:red;font-weight:700;text-decoration:line-through;">like alcohol and tobacco while criminalizing the use of </span><span style="color:red;font-weight:700;">are allowed, then it is reasonable to legalize </span>cannabis, which is less harmful. <span style="color:red;font-weight:700;text-decoration:line-through;">Legalizing cannabis </span><span style="color:red;font-weight:700;">Furthermore, legalization </span>would <span style="color:red;font-weight:700;text-decoration:line-through;">also </span>allow for regulation and oversight by the <span style="color:red;font-weight:700;text-decoration:line-through;">FDA and </span>government, which is currently lacking in the <span style="color:red;font-weight:700;text-decoration:line-through;">unregulated </span>cannabis market.


ideology-content VS. all
------------------------


<span style="color:red;font-weight:700;text-decoration:line-through;">As individuals, we should </span><span style="color:red;font-weight:700;">Individuals </span>have the <span style="color:red;font-weight:700;text-decoration:line-through;">freedom </span><span style="color:red;font-weight:700;">right </span>to make <span style="color:red;font-weight:700;text-decoration:line-through;">choices </span><span style="color:red;font-weight:700;">decisions </span>about <span style="color:red;font-weight:700;text-decoration:line-through;">our </span><span style="color:red;font-weight:700;">their </span>own bodies without <span style="color:red;font-weight:700;text-decoration:line-through;">interference from the government. If we are not harming anyone else, there is no reason to prevent us from enjoying ourselves. Additionally, it is hypocritical to allow the consumption of more harmful substances like alcohol and tobacco while criminalizing </span><span style="color:red;font-weight:700;">interference. This includes </span>the use of cannabis, which is less <span style="color:red;font-weight:700;text-decoration:line-through;">harmful. </span><span style="color:red;font-weight:700;">harmful than other legal substances like alcohol and tobacco. </span>Legalizing cannabis would <span style="color:red;font-weight:700;text-decoration:line-through;">also </span>allow for regulation and oversight by the <span style="color:red;font-weight:700;text-decoration:line-through;">FDA and </span>government, which is currently lacking in the unregulated <span style="color:red;font-weight:700;text-decoration:line-through;">cannabis </span>market.


ideology-style VS. all
----------------------


Individuals have the right to make <span style="color:red;font-weight:700;text-decoration:line-through;">choices regarding </span><span style="color:red;font-weight:700;">decisions about </span>their own bodies without interference. <span style="color:red;font-weight:700;text-decoration:line-through;">These choices do not harm others, and therefore should not be restricted. Additionally, it is worth noting that there are other substances, such as alcohol and tobacco, which are legal despite being more harmful than cannabis. If these substances are allowed, then it is reasonable to legalize </span><span style="color:red;font-weight:700;">This includes the use of </span>cannabis, which is less <span style="color:red;font-weight:700;text-decoration:line-through;">harmful. Furthermore, legalization </span><span style="color:red;font-weight:700;">harmful than other legal substances like alcohol and tobacco. Legalizing cannabis </span>would allow for regulation and oversight by the government, which is currently lacking in the <span style="color:red;font-weight:700;text-decoration:line-through;">cannabis </span><span style="color:red;font-weight:700;">unregulated </span>market.

In [ ]:
for k, v in result_dict.items():
    for k,v  in result_dict.items():
    print(f"\n{k} - {prompt_dict[k]}")
    diff = Redlines(ineffective_argument,v)
    display(Markdown(diff.output_markdown))